# TC-Impact on Met: Weather Stations

Here we search the GSOD for weather stations near TC landfall to assess the impact of TC passage on the surface meteorology. 

The idea is to search for the nearest station that has ALL data present for landfall +/-30 days, and has at least k % completion for j years, and l % completion for the period +/- landfall within that ten-year period 

In [374]:
import ulmo as ul, numpy as np, pandas as pd,TC_Utils as tc, \
GeneralFunctions as GF, datetime as datetime, matplotlib.pyplot as plt, os

In [370]:
# Set parameters
minlen=10 # years of observations
minpc=70 # percentage complete over minlen period
maxdist=197 # maximum distance to search away from the TC (radius to 12 m/s wind [Chavas and Emmanuel, 2010])
min_frac=50.0 # GSOD Data be at least this complete to count as "complete (over 1981-2010)"

# Open the obs file (with landfall locations)
# strcture is: ID	YEAR	JD.JD	LAT	LON	GLAT	GLON	ROW	COL	DIST
tcdata=np.loadtxt("/media/gytm3/WD12TB/TropicalCyclones/TC-DeadlyHeat/Data/LandFall.txt",delimiter="\t",skiprows=1)

# Pull out landfall lat/lon
tclat=tcdata[:,3]; tclon=tcdata[:,4]
# Set the output file name. It will be a log with ID | LAT | LON | STATION ID | Distance
outfile="/media/gytm3/WD12TB/TropicalCyclones/TC-DeadlyHeat/Data/GSOD_Stations.txt"
# Set the output directory to write the "complete GSOD data to"
odir="/media/gytm3/WD12TB/TropicalCyclones/TC-DeadlyHeat/Data/GSOD_Complete/"

In [282]:
# Use ulmo to return meta data on ALL stations with data since 1979
all_stations=ul.ncdc.gsod.get_stations(start="1979-01-01 00:00:00")

## Find stations that are close enough to TC landfall (within maxdist) and have SLP & HI data for the TC landfall dat (+/- 30 days)

In [334]:
# Use ulmo-modified code to extract met data for the station nearest the TC landfall
reload(tc)
ids=all_stations.keys() # station codes
latlon=np.array([[all_stations[ii]["latitude"],all_stations[ii]["longitude"]] for ii in ids],dtype=np.float) # coordinates
# Test now by extracting pressure 
outmet=[]
outmet_hit=[]
out_station=[]
dist_used=np.zeros((len(tcdata)))*np.nan
nsuc=0
with open(outfile,"w") as fo:
    
    # Write the header to the output file
    fo.write("YEAR\tMONTH\tDAY\tID\tLAT\tLON\tSTATION_ID\tDIST\n")
    
    for loc in range(len(tcdata)):

        # If all criteria are met, we'll not search any more; initialise to "search"
        dist=GF.haversine_fast(tclat[loc],tclon[loc],latlon[:,0],latlon[:,1],miles=False) # that's the distance
        order=np.argsort(dist)
        nearest=0
        mindist=dist[order[nearest]]
        keep_search=True
        while keep_search and mindist<maxdist:

            # Construct date index for this landfall location
            tc_date=datetime.date(year=int(tcdata[loc,1]),month=1,day=1)+datetime.timedelta(days=tcdata[loc,2]-1)
            tc_st=tc_date-datetime.timedelta(days=30)
            tc_end=tc_date+datetime.timedelta(days=30)

            # Now extract all data for the year (+/-1) of TC landfall for the station closest to landfall location
            startyr=tc_date.year-1; endyr=tc_date.year+1
            alldata,corevars=tc.get_gsod_data("/media/gytm3/WD12TB/GSOD",\
            [ids[order[nearest]],],startyr=int(startyr), endyr=int(endyr), parameters=None)

            # Check we have something!
            if len(corevars)>1:     

                # Compute HI. etc
                corevars["tempC"]=GF.Faren2C(corevars["mean_temp"])
                corevars["dewC"]=GF.Faren2C(corevars["dew_point"])
                corevars["rh"]=GF.dewVp(corevars["dewC"]+273.15)/GF.satVp(corevars["tempC"])*100.            
                corevars["specHum"]=GF.specHum(corevars["tempC"],corevars["rh"],corevars["slp"])
                corevars["hi"]=GF.HeatIndexNWS(corevars["tempC"],corevars["rh"],opt=False,iounit=np.array([0,0]))

                # Check for completion during "hit"
                hit_data=corevars[tc_st:tc_end]

                # Examine pressure and hi
                # Note these ranges should be constrained based on GSOD meta-data
                n_press=np.sum(np.logical_and(hit_data["slp"]<1100,hit_data["slp"]>800))
                n_hi=np.sum(np.logical_and(hit_data["hi"]>-100,hit_data["hi"]<200))

                # Then all data here
                if n_press == 61 and n_hi == 61:
                    outmet.append(corevars)
                    outmet_hit.append(hit_data)
                    dist_used[loc]=dist[order[nearest]]
                    out_station.append(ids[order[nearest]])

                    # End search
                    keep_search=False; print "Found what we're after! [dist=%.1f; station=%s]" \
                    % (dist[order[nearest]],ids[order[nearest]])
                    nsuc+=1
                    
                    # Write a new line out
                    lout="%.0f\t%.0f\t%.0f\t%.0f\t%.2f\t%.2f\t%s\t%.2f\n" % \
                    (tc_date.year,tc_date.month,tc_date.day,\
                     tcdata[loc,0],tcdata[loc,3],tcdata[loc,4],ids[order[nearest]],dist[order[nearest]])
                    fo.write(lout); fo.flush()

            if keep_search:
                mindist=dist[order[nearest+1]]
                print "Didn't satisfy criteria [dist=%.1f]; moving to distance: %.1f" % \
                    (dist[order[nearest]],mindist)


            nearest+=1  
        print "Finished with loc %.0f" % loc
            





Didn't satisfy criteria [dist=67.8]; moving to distance: 133.0
Didn't satisfy criteria [dist=133.0]; moving to distance: 134.1
Didn't satisfy criteria [dist=134.1]; moving to distance: 214.2
Finished with loc 0
Didn't satisfy criteria [dist=14.5]; moving to distance: 16.4
Didn't satisfy criteria [dist=16.4]; moving to distance: 99.5
Didn't satisfy criteria [dist=99.5]; moving to distance: 103.3
Didn't satisfy criteria [dist=103.3]; moving to distance: 194.2
Didn't satisfy criteria [dist=194.2]; moving to distance: 225.0
Finished with loc 1
Didn't satisfy criteria [dist=57.5]; moving to distance: 89.3
Didn't satisfy criteria [dist=89.3]; moving to distance: 122.7
Didn't satisfy criteria [dist=122.7]; moving to distance: 123.6
Didn't satisfy criteria [dist=123.6]; moving to distance: 159.5
Didn't satisfy criteria [dist=159.5]; moving to distance: 167.5
Didn't satisfy criteria [dist=167.5]; moving to distance: 189.3
Didn't satisfy criteria [dist=189.3]; moving to distance: 218.6
Finished 

Didn't satisfy criteria [dist=113.0]; moving to distance: 120.0
Didn't satisfy criteria [dist=120.0]; moving to distance: 140.6
Didn't satisfy criteria [dist=140.6]; moving to distance: 140.8
Didn't satisfy criteria [dist=140.8]; moving to distance: 151.5
Didn't satisfy criteria [dist=151.5]; moving to distance: 190.6
Didn't satisfy criteria [dist=190.6]; moving to distance: 198.2
Finished with loc 20
Didn't satisfy criteria [dist=18.6]; moving to distance: 71.5
Didn't satisfy criteria [dist=71.5]; moving to distance: 71.9
Didn't satisfy criteria [dist=71.9]; moving to distance: 90.4
Didn't satisfy criteria [dist=90.4]; moving to distance: 92.1
Didn't satisfy criteria [dist=92.1]; moving to distance: 95.7
Didn't satisfy criteria [dist=95.7]; moving to distance: 98.0
Found what we're after! [dist=98.0; station=984400-99999]
Finished with loc 21
Didn't satisfy criteria [dist=109.8]; moving to distance: 111.5
Didn't satisfy criteria [dist=111.5]; moving to distance: 120.9
Didn't satisfy c

Didn't satisfy criteria [dist=99.6]; moving to distance: 118.9
Didn't satisfy criteria [dist=118.9]; moving to distance: 119.5
Didn't satisfy criteria [dist=119.5]; moving to distance: 128.0
Didn't satisfy criteria [dist=128.0]; moving to distance: 138.3
Didn't satisfy criteria [dist=138.3]; moving to distance: 151.2
Didn't satisfy criteria [dist=151.2]; moving to distance: 153.0
Didn't satisfy criteria [dist=153.0]; moving to distance: 178.3
Didn't satisfy criteria [dist=178.3]; moving to distance: 186.0
Didn't satisfy criteria [dist=186.0]; moving to distance: 187.5
Didn't satisfy criteria [dist=187.5]; moving to distance: 208.9
Finished with loc 34
Didn't satisfy criteria [dist=34.3]; moving to distance: 43.0
Didn't satisfy criteria [dist=43.0]; moving to distance: 52.4
Didn't satisfy criteria [dist=52.4]; moving to distance: 52.4
Didn't satisfy criteria [dist=52.4]; moving to distance: 60.5
Didn't satisfy criteria [dist=60.5]; moving to distance: 60.9
Didn't satisfy criteria [dist=

Didn't satisfy criteria [dist=39.7]; moving to distance: 42.1
Didn't satisfy criteria [dist=42.1]; moving to distance: 42.1
Found what we're after! [dist=42.1; station=747770-03852]
Finished with loc 55
Didn't satisfy criteria [dist=21.1]; moving to distance: 24.3
Didn't satisfy criteria [dist=24.3]; moving to distance: 61.0
Didn't satisfy criteria [dist=61.0]; moving to distance: 61.1
Didn't satisfy criteria [dist=61.1]; moving to distance: 110.4
Didn't satisfy criteria [dist=110.4]; moving to distance: 111.1
Didn't satisfy criteria [dist=111.1]; moving to distance: 124.2
Found what we're after! [dist=124.2; station=984440-99999]
Finished with loc 56
Didn't satisfy criteria [dist=24.6]; moving to distance: 37.4
Didn't satisfy criteria [dist=37.4]; moving to distance: 59.7
Didn't satisfy criteria [dist=59.7]; moving to distance: 63.9
Didn't satisfy criteria [dist=63.9]; moving to distance: 72.2
Didn't satisfy criteria [dist=72.2]; moving to distance: 84.9
Didn't satisfy criteria [dist=

Found what we're after! [dist=136.9; station=943000-99999]
Finished with loc 81
Found what we're after! [dist=9.3; station=478380-99999]
Finished with loc 82
Didn't satisfy criteria [dist=50.6]; moving to distance: 81.4
Didn't satisfy criteria [dist=81.4]; moving to distance: 91.8
Didn't satisfy criteria [dist=91.8]; moving to distance: 96.9
Didn't satisfy criteria [dist=96.9]; moving to distance: 104.5
Didn't satisfy criteria [dist=104.5]; moving to distance: 188.5
Didn't satisfy criteria [dist=188.5]; moving to distance: 190.1
Didn't satisfy criteria [dist=190.1]; moving to distance: 190.8
Didn't satisfy criteria [dist=190.8]; moving to distance: 205.5
Finished with loc 83
Didn't satisfy criteria [dist=33.1]; moving to distance: 34.7
Didn't satisfy criteria [dist=34.7]; moving to distance: 55.9
Didn't satisfy criteria [dist=55.9]; moving to distance: 61.0
Didn't satisfy criteria [dist=61.0]; moving to distance: 84.9
Didn't satisfy criteria [dist=84.9]; moving to distance: 127.2
Didn'

...

Didn't satisfy criteria [dist=82.4]; moving to distance: 82.4
Didn't satisfy criteria [dist=82.4]; moving to distance: 102.8
Didn't satisfy criteria [dist=102.8]; moving to distance: 103.6
Didn't satisfy criteria [dist=103.6]; moving to distance: 103.6
Found what we're after! [dist=103.6; station=591580-99999]
Finished with loc 120
Didn't satisfy criteria [dist=15.3]; moving to distance: 15.7
Didn't satisfy criteria [dist=15.7]; moving to distance: 16.7
Didn't satisfy criteria [dist=16.7]; moving to distance: 16.7
Didn't satisfy criteria [dist=16.7]; moving to distance: 18.2
Didn't satisfy criteria [dist=18.2]; moving to distance: 46.9
Didn't satisfy criteria [dist=46.9]; moving to distance: 46.9
Didn't satisfy criteria [dist=46.9]; moving to distance: 48.7
Didn't satisfy criteria [dist=48.7]; moving to distance: 49.3
Didn't satisfy criteria [dist=49.3]; moving to distance: 146.1
Didn't satisfy criteria [dist=146.1]; moving to distance: 171.6
Didn't satisfy criteria [dist=171.6]; movin

Found what we're after! [dist=110.3; station=670950-99999]
Finished with loc 136
Didn't satisfy criteria [dist=6.5]; moving to distance: 31.3
Didn't satisfy criteria [dist=31.3]; moving to distance: 39.3
Didn't satisfy criteria [dist=39.3]; moving to distance: 98.8
Didn't satisfy criteria [dist=98.8]; moving to distance: 126.3
Didn't satisfy criteria [dist=126.3]; moving to distance: 131.7
Didn't satisfy criteria [dist=131.7]; moving to distance: 134.1
Didn't satisfy criteria [dist=134.1]; moving to distance: 135.3
Didn't satisfy criteria [dist=135.3]; moving to distance: 136.0
Didn't satisfy criteria [dist=136.0]; moving to distance: 141.6
Didn't satisfy criteria [dist=141.6]; moving to distance: 147.2
Didn't satisfy criteria [dist=147.2]; moving to distance: 152.2
Didn't satisfy criteria [dist=152.2]; moving to distance: 152.2
Didn't satisfy criteria [dist=152.2]; moving to distance: 152.7
Didn't satisfy criteria [dist=152.7]; moving to distance: 154.3
Didn't satisfy criteria [dist=1

Didn't satisfy criteria [dist=114.9]; moving to distance: 115.1
Found what we're after! [dist=115.1; station=952860-99999]
Finished with loc 144
Found what we're after! [dist=85.4; station=982320-99999]
Finished with loc 145
Didn't satisfy criteria [dist=81.0]; moving to distance: 142.8
Found what we're after! [dist=142.8; station=982330-99999]
Finished with loc 146
Didn't satisfy criteria [dist=54.7]; moving to distance: 91.0
Found what we're after! [dist=91.0; station=943100-99999]
Finished with loc 147
Didn't satisfy criteria [dist=47.3]; moving to distance: 92.0
Didn't satisfy criteria [dist=92.0]; moving to distance: 108.9
Found what we're after! [dist=108.9; station=943100-99999]
Finished with loc 148
Didn't satisfy criteria [dist=2.8]; moving to distance: 3.1
Didn't satisfy criteria [dist=3.1]; moving to distance: 3.2
Didn't satisfy criteria [dist=3.2]; moving to distance: 5.8
Didn't satisfy criteria [dist=5.8]; moving to distance: 15.4
Didn't satisfy criteria [dist=15.4]; movin

Didn't satisfy criteria [dist=72.2]; moving to distance: 84.8
Didn't satisfy criteria [dist=84.8]; moving to distance: 86.1
Didn't satisfy criteria [dist=86.1]; moving to distance: 98.8
Didn't satisfy criteria [dist=98.8]; moving to distance: 127.2
Didn't satisfy criteria [dist=127.2]; moving to distance: 129.3
Didn't satisfy criteria [dist=129.3]; moving to distance: 134.0
Found what we're after! [dist=134.0; station=916800-99999]
Finished with loc 171
Finished with loc 172
Found what we're after! [dist=15.2; station=983340-99999]
Finished with loc 173
Found what we're after! [dist=24.5; station=982330-99999]
Finished with loc 174
Didn't satisfy criteria [dist=23.8]; moving to distance: 40.7
Didn't satisfy criteria [dist=40.7]; moving to distance: 47.1
Found what we're after! [dist=47.1; station=984440-99999]
Finished with loc 175
Didn't satisfy criteria [dist=16.2]; moving to distance: 32.2
Didn't satisfy criteria [dist=32.2]; moving to distance: 33.0
Didn't satisfy criteria [dist=33

Didn't satisfy criteria [dist=33.2]; moving to distance: 41.3
Didn't satisfy criteria [dist=41.3]; moving to distance: 42.3
Didn't satisfy criteria [dist=42.3]; moving to distance: 43.2
Didn't satisfy criteria [dist=43.2]; moving to distance: 43.8
Didn't satisfy criteria [dist=43.8]; moving to distance: 44.0
Didn't satisfy criteria [dist=44.0]; moving to distance: 51.1
Didn't satisfy criteria [dist=51.1]; moving to distance: 51.2
Didn't satisfy criteria [dist=51.2]; moving to distance: 51.2
Didn't satisfy criteria [dist=51.2]; moving to distance: 59.4
Didn't satisfy criteria [dist=59.4]; moving to distance: 61.5
Didn't satisfy criteria [dist=61.5]; moving to distance: 61.7
Didn't satisfy criteria [dist=61.7]; moving to distance: 64.9
Didn't satisfy criteria [dist=64.9]; moving to distance: 65.4
Didn't satisfy criteria [dist=65.4]; moving to distance: 78.0
Didn't satisfy criteria [dist=78.0]; moving to distance: 81.3
Didn't satisfy criteria [dist=81.3]; moving to distance: 93.5
Didn't s

## Assess the impact of TC passage on the met recorded at these proximate weather stations


### Start with the "gold standard": those stations close to TC landfall AND with long climatology

In [375]:
# Read in the metadata. All those TC landfall locations within maxdist of a (functioning) AWS are listed under
# "STATION_ID".

# We will take this list of functioning AWSs and further filter based on data availability over 
# the 1981-2010 period
metadata=pd.read_csv(outfile,delimiter="\t")
# Days between dates
st=datetime.datetime(year=1981,month=1,day=1); end=datetime.datetime(year=2010,month=12,day=31)
nd=(end-st).total_seconds()/(60**2*24.)+1
ncom=0 + len([ii for ii in os.listdir(odir) if ".csv" in ii])
for station in metadata["STATION_ID"]:
    if station + ".csv" not in os.listdir(odir):
        all,corevars=tc.get_gsod_data("/media/gytm3/WD12TB/GSOD",\
                [station,],startyr=int(1981), endyr=int(2010), \
                         parameters=["mean_temp","dew_point","sea_level_pressure"])
        corevars["tempC"]=GF.Faren2C(corevars["mean_temp"])
        corevars["dewC"]=GF.Faren2C(corevars["dew_point"])
        corevars["rh"]=GF.dewVp(corevars["dewC"]+273.15)/GF.satVp(corevars["tempC"])*100.            
        corevars["specHum"]=GF.specHum(corevars["tempC"],corevars["rh"],corevars["slp"])
        corevars["hi"]=GF.HeatIndexNWS(corevars["tempC"],corevars["rh"],opt=False,iounit=np.array([0,0]))
        
        # Get frac complete
        frac=np.sum(~np.isnan(corevars[["hi","slp"]].sum(axis=1)))/nd*100.
        print "station %s is %0.f%% complete" % (station,frac)
        if frac >= min_frac:
            ncom+=1
            corevars.to_csv(odir+station+".csv",float_format="%.2f"); print "written file: %.0f"%ncom
        
    

station 943100-99999 is 42% complete
station 943100-99999 is 42% complete
station 953170-99999 is 26% complete
station 953170-99999 is 26% complete
station 747686-99999 is 23% complete
station 747686-99999 is 23% complete
station 747686-99999 is 23% complete
station 941390-99999 is 40% complete
station 983300-99999 is 49% complete
station 953170-99999 is 26% complete
station 943100-99999 is 42% complete
station 952860-99999 is 27% complete
station 943100-99999 is 42% complete
station 943100-99999 is 42% complete
station 983340-99999 is 28% complete
station 985380-99999 is 90% complete
written file: 47
station 984270-99999 is 86% complete
written file: 48
station 597580-99999 is 100% complete
written file: 49
station 985530-99999 is 32% complete
station 941400-99999 is 64% complete
written file: 50
station 953700-99999 is 17% complete
station 953700-99999 is 17% complete
station 983340-99999 is 28% complete
station 983340-99999 is 28% complete
station 766540-99999 is 83% complete
writte

In [364]:
ncom=np.sum(~np.isnan(corevars[["hi","slp"]].sum(axis=1)))

In [366]:
ncom; nd

10957.0